### Week 2 Day 5 exercise: The Football Banter Bot 
This exercise is a conversation between two specialized Large Language Models (LLMs) programmed with conflicting personas. Utilizing a state-machine architecture, the system pits a Snarky Chelsea Supporter (powered by Claude 3.5 Sonnet) against a Brusque Manchester City Supporter (powered by GPT-4o Mini) to argue about their teams.

In [ ]:
import os
import gradio as gr
import time
from dotenv import load_dotenv
from openai import OpenAI


In [ ]:
# Load environment variables
load_dotenv(override=True)
openrouter_url = "https://openrouter.ai/api/v1"
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")

In [ ]:
# Initialize
openrouter = OpenAI(base_url=openrouter_url, api_key=openrouter_api_key)
ANTROPIC_CHELSEA_MODEL = "anthropic/claude-3.5-sonnet"
MAN_CITY_MODEL = "openai/gpt-4o-mini"


In [ ]:
# Persona Prompt Instructions
CHELSEA_PERSONA_PROMPT = """ 
You are a snarky, condescending Chelsea fan. Use sophisticated wit. 
Mock Man City for having no fans, being a 'state-owned' project, and having 115 charges.
Keep responses to 2-3 sentences.
"""
MAN_CITY_PERSONA_PROMPT = """  
You are a brusque, blunt Man City fan. You are bored by Chelsea's 'history' talk.
Focus on the 4-in-a-row titles and Chelsea being a mid-table circus.
Be short and rude. Keep responses to 1-2 sentences.
"""

In [ ]:
def kick_off_beef(messages, model):
    response_stream = openrouter.chat.completions.create(
        model=model,
        messages=messages,
        stream=True
    )
    response = ""
    for chunk in response_stream:
        response += chunk.choices[0].delta.content or ""
        yield response

In [ ]:
def beef_debate(chat_history):
    def llm_messages(history):
        msgs = []
        for msg in history:
            role = "assistant" if msg["role"] == "assistant" else "user"
            msgs.append({"role": role, "content": msg["content"]})
        return msgs

    # Run for 3 rounds
    for i in range(3):
        is_last_round = i == 2

        # 1. Chelsea's Turn
        chelsea_instructions = CHELSEA_PERSONA_PROMPT
        if is_last_round:
            chelsea_instructions += " CONCEDE NOW. Admit City is technically better right now, but remain bitter."

        messages = [{"role": "system", "content": chelsea_instructions}] + llm_messages(
            chat_history
        )

        response = ""
        for partial in kick_off_beef(messages, ANTROPIC_CHELSEA_MODEL):
            response = partial

        chat_history.append({"role": "assistant", "content": f"🦁 **Chelsea**: {response}"})
        yield chat_history
        time.sleep(1)

        if is_last_round:
            break  # Stop once Chelsea concedes

        # 2. City's Turn
        messages = [{"role": "system", "content": MAN_CITY_PERSONA_PROMPT}] + llm_messages(chat_history)

        response = ""
        for partial in kick_off_beef(messages, MAN_CITY_MODEL):
            response = partial

        chat_history.append({"role": "assistant", "content": f"🩵 **City**: {response}"})
        yield chat_history
        time.sleep(1)

In [ ]:
# --- GRADIO LIVE INTERFACE ---

with gr.Blocks() as rivary_debate:
    gr.Markdown("## ⚽ The Football Banter Bot ")

    chatbot = gr.Chatbot(label="The Banter Battle", height=600, type="messages")
    with gr.Row():
        start_btn = gr.Button("Start Beef", variant="primary")
        clear_btn = gr.Button("Clear")

    start_btn.click(beef_debate, inputs=[chatbot], outputs=[chatbot])
    clear_btn.click(lambda: [], None, chatbot)

rivary_debate.launch(inbrowser=True)